In [5]:
#!pip install opencv-python numpy pandas facenet-pytorch torch torchvision onnxruntime

In [1]:
import cv2
import numpy as np
import pandas as pd
from facenet_pytorch import MTCNN
import os
import torch
from datetime import datetime
import torchvision.transforms as transforms


# Initialize face detector
mtcnn = MTCNN(keep_all=False)  # Only 1 face at a time

# Output CSV file
CSV_FILE = 'face_dataset1.csv'

# Create CSV if not exists
if not os.path.exists(CSV_FILE):
    df = pd.DataFrame(columns=["name"] + [f"emb_{i}" for i in range(128)])
    df.to_csv(CSV_FILE, index=False)


In [ ]:
name = input("Enter the person's name: ")
cap = cv2.VideoCapture(0)

count = 0
MAX_SAMPLES = 10

while count < MAX_SAMPLES:
    ret, frame = cap.read()
    if not ret:
        break

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes, probs = mtcnn.detect(img)

    if boxes is not None:
        box = boxes[0]  # Only first face
        x1, y1, x2, y2 = [int(v) for v in box]
        face = img[y1:y2, x1:x2]
        if face.size == 0:
            continue
        face = cv2.resize(face, (112, 112))

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.imshow("Face", cv2.cvtColor(face, cv2.COLOR_RGB2BGR))

    cv2.imshow("Webcam", frame)
    key = cv2.waitKey(1)

    if key & 0xFF == ord('c') and boxes is not None:
        face_tensor = transforms.ToTensor()(face).unsqueeze(0) 
        face_tensor = face_tensor.float()  
        with torch.no_grad():
            embedding = model(face_tensor).squeeze().numpy()


        
        row = [name] + embedding.tolist()
        pd.DataFrame([row]).to_csv(CSV_FILE, mode='a', index=False, header=False)
        count += 1
        print(f"[{count}] Sample captured for {name}")

    elif key & 0xFF == ord('q'):
        print("Exit requested.")
        break

cap.release()
cv2.destroyAllWindows()
